In [2]:
import cv2
import mediapipe as mp
import numpy as np
import random
import time

# Adjustable parameters
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600
BALL_RADIUS = 30
BALL_SPEED_RANGE = (5, 12)            # Min/max ball speed (pixels/frame)
SPAWN_INTERVAL = 0.6                  # Seconds between ball spawns
BALL_COLORS = {"red": (0, 0, 255), "green": (0, 255, 0), "blue": (255, 0, 0)}
BALL_POINTS = {"red": 10, "green": 20, "blue": 50}
BALL_PROBS = {"red": 0.5, "green": 0.4, "blue": 0.1}

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.7,
                       min_tracking_confidence=0.7)

font = cv2.FONT_HERSHEY_SIMPLEX

class Ball:
    def __init__(self):
        self.color = random.choices(list(BALL_COLORS.keys()), weights=list(BALL_PROBS.values()))[0]
        self.x = random.randint(BALL_RADIUS, WINDOW_WIDTH - BALL_RADIUS)
        self.y = -BALL_RADIUS
        self.speed = random.randint(BALL_SPEED_RANGE, BALL_SPEED_RANGE)
        self.radius = BALL_RADIUS
        self.points = BALL_POINTS[self.color]
        self.rgb = BALL_COLORS[self.color]

    def move(self):
        self.y += self.speed

    def draw(self, frame):
        cv2.circle(frame, (self.x, int(self.y)), self.radius, self.rgb, -1)

    def is_off_screen(self):
        return self.y - self.radius > WINDOW_HEIGHT

def distance(pt1, pt2):
    return np.sqrt((pt1[0] - pt2)**2 + (pt1 - pt2)**2)

def show_start_screen(frame, button_coords, button_radius):
    overlay = frame.copy()
    cv2.circle(overlay, button_coords, button_radius, (0, 255, 255), -1)
    cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
    cv2.putText(frame, "START GAME", (button_coords[0]-110, button_coords+10),
                font, 1.4, (0, 0, 0), 5)

def show_game_over(frame, score):
    cv2.rectangle(frame, (100, 200), (WINDOW_WIDTH-100, WINDOW_HEIGHT-200), (50, 50, 50), -1)
    cv2.putText(frame, "GAME OVER", (WINDOW_WIDTH//2 - 170, WINDOW_HEIGHT//2 - 60),
                font, 2.3, (0, 0, 255), 6)
    cv2.putText(frame, f"Final Score: {score}", (WINDOW_WIDTH//2 - 170, WINDOW_HEIGHT//2 + 30),
                font, 1.6, (0, 255, 255), 4)

def main():
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, WINDOW_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, WINDOW_HEIGHT)

    balls = []
    last_spawn_time = 0
    score = 0
    missed = 0
    running = False
    fps_time = time.time()
    start_button_center = (WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2)
    start_button_radius = 80

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (WINDOW_WIDTH, WINDOW_HEIGHT))
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)
        index_finger_tip = None

        # Get index finger tip coordinates if a hand is detected
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Landmark 8 is index fingertip
                x = int(hand_landmarks.landmark[8].x * WINDOW_WIDTH)
                y = int(hand_landmarks.landmark.y * WINDOW_HEIGHT)
                index_finger_tip = (x, y)
                cv2.circle(frame, index_finger_tip, 16, (255, 255, 0), -1)
                break

        if not running:
            show_start_screen(frame, start_button_center, start_button_radius)
            if index_finger_tip is not None:
                # Detect fingertip collision with start button
                if distance(index_finger_tip, start_button_center) < start_button_radius:
                    running = True
                    balls = []
                    last_spawn_time = time.time()
                    score = 0
                    missed = 0
                    time.sleep(0.5)  # Simple debounce

        elif missed < 3:
            # Spawn new balls
            if time.time() - last_spawn_time > SPAWN_INTERVAL:
                balls.append(Ball())
                last_spawn_time = time.time()

            # Move and draw balls
            for ball in balls:
                ball.move()
                ball.draw(frame)

            # Check collisions
            caught_balls = []
            if index_finger_tip is not None:
                for ball in balls:
                    if distance(index_finger_tip, (ball.x, int(ball.y))) < ball.radius:
                        score += ball.points
                        caught_balls.append(ball)
            # Remove caught balls
            balls = [ball for ball in balls if ball not in caught_balls]

            # Balls missed
            missed_balls = [ball for ball in balls if ball.is_off_screen()]
            missed += len(missed_balls)
            balls = [ball for ball in balls if not ball.is_off_screen()]

            # Overlay score and missed count
            cv2.putText(frame, f"Score: {score}", (24, 50), font, 1.2, (255, 255, 255), 3)
            cv2.putText(frame, f"Missed: {missed}/3", (24, 96), font, 1.2, (0, 0, 255), 3)

        else:
            show_game_over(frame, score)
            cv2.putText(frame, "Press R to restart", (WINDOW_WIDTH//2 - 170, WINDOW_HEIGHT//2 + 100),
                        font, 1.0, (255, 255, 255), 2)

        # FPS display (for tuning/diagnostics)
        elapsed = time.time() - fps_time
        fps = int(1/elapsed) if elapsed > 0 else 0
        fps_time = time.time()
        cv2.putText(frame, f"{fps} FPS", (WINDOW_WIDTH-160, 44), font, 1.0, (0,255,255), 2)

        cv2.imshow("Fingertip Catch Game", frame)

        # Key press handler
        key = cv2.waitKey(1) & 0xFF
        if key == 27:      # ESC to quit
            break
        if key == ord('r'):  # 'r' to restart after game over
            running = False

    cap.release()
    cv2.destroyAllWindows()
    hands.close()

if __name__ == "__main__":
    main()


TypeError: can only concatenate tuple (not "int") to tuple